In [1]:
from types import SimpleNamespace
from functools import lru_cache
import os
import time
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np
import scipy.io.wavfile
import scipy.fftpack
import scipy.linalg
import torch
import torch.utils.data as data
import torch.nn as nn
import torch.optim as optim
import math

In [2]:
@lru_cache(maxsize=10)
def get_window(n, type='hamming'):
    coefs = np.arange(n)
    window = 0.54 - 0.46 * np.cos(2 * np.pi * coefs / (n - 1))
    return window

def apply_preemphasis(y, preemCoef=0.97):
    y[1:] = y[1:] - preemCoef*y[:-1]
    y[0] *= (1 - preemCoef)
    return y

def freq_to_mel(freq):
    return 2595.0 * np.log10(1.0 + freq / 700.0)

def mel_to_freq(mels):
    return 700.0 * (np.power(10.0, mels / 2595.0) - 1.0)

@lru_cache(maxsize=10)
def get_filterbank(numfilters, filterLen, lowFreq, highFreq, samplingFreq):
    minwarpfreq = freq_to_mel(lowFreq)
    maxwarpfreq = freq_to_mel(highFreq)
    dwarp = (maxwarpfreq - minwarpfreq) / (numfilters + 1)
    f = mel_to_freq(np.arange(numfilters + 2) * dwarp + minwarpfreq) * (filterLen - 1) * 2.0 / samplingFreq
    i = np.arange(filterLen)[None, :]
    f = f[:, None]
    hislope = (i - f[:numfilters]) / (f[1:numfilters+1] - f[:numfilters])
    loslope = (f[2:numfilters+2] - i) / (f[2:numfilters+2] - f[1:numfilters+1])
    H = np.maximum(0, np.minimum(hislope, loslope))
    return H

def normalized(y, threshold=0):
    y -= y.mean()
    stddev = y.std()
    if stddev > threshold:
        y /= stddev
    return y

def mfsc(y, sfr, window_size=0.025, window_stride=0.010, window='hamming', normalize=True, log=True, n_mels=80, preemCoef=0.97, melfloor=1.0):
    win_length = int(sfr * window_size)
    hop_length = int(sfr * window_stride)
    n_fft = 2048
    lowfreq = 0
    highfreq = sfr/2
    
    # get window
    window = get_window(win_length)
    padded_window = np.pad(window, (0, n_fft - win_length), mode='constant')[:, None]
    
    # preemphasis
    y = apply_preemphasis(y, preemCoef)

    # scale wave signal
    y *= 32768
    
    # get frames and scale input
    num_frames = 1 + (len(y) - win_length) // hop_length
    pad_after = num_frames*hop_length + (n_fft - hop_length) - len(y)
    if pad_after > 0:
        y = np.pad(y, (0, pad_after), mode='constant')
    frames = np.lib.stride_tricks.as_strided(y, shape=(n_fft, num_frames), strides=(y.itemsize, hop_length * y.itemsize), writeable=False)
    windowed_frames = padded_window * frames
    D = np.abs(np.fft.rfft(windowed_frames, axis=0))

    # mel filterbank
    filterbank = get_filterbank(n_mels, n_fft/2 + 1, lowfreq, highfreq, sfr)
    mf = np.dot(filterbank, D)
    mf = np.maximum(melfloor, mf)
    if log:
        mf = np.log(mf)
    if normalize:
        mf = normalized(mf)

    return mf

In [3]:
def make_dataset(kaldi_path, class_to_id):
    text_path = os.path.join(kaldi_path, 'text')
    wav_path = os.path.join(kaldi_path, 'wav.scp')

    key_to_word = dict()
    key_to_wav = dict()
    
    with open(wav_path, 'rt') as wav_scp:
        for line in wav_scp:
            key, wav = line.strip().split(' ', 1)
            key_to_wav[key] = wav
            key_to_word[key] = None # default

    if os.path.isfile(text_path):
        with open(text_path, 'rt') as text:
            for line in text:
                key, word = line.strip().split(' ', 1)
                key_to_word[key] = word

    wavs = []
    for key, wav_command in key_to_wav.items():
        word = key_to_word[key]
        word_id = class_to_id[word] if word is not None else -1 # default for test
        wav_item = [key, wav_command, word_id]
        wavs.append(wav_item)

    return wavs

In [4]:
def wav_read(path):
    sr, y = scipy.io.wavfile.read(path)
    y = y/32768 # Normalize to -1..1
    y -= y.mean()
    return y, sr

In [5]:
def param_loader(path, window_size, window_stride, window, normalize, max_len):
    y, sfr = wav_read(path)

    param = mfsc(y, sfr, window_size=window_size, window_stride=window_stride, window=window, normalize=normalize, log=False, n_mels=60, preemCoef=0, melfloor=1.0)

    # Add zero padding to make all param with the same dims
    if param.shape[1] < max_len:
        pad = np.zeros((param.shape[0], max_len - param.shape[1]))
        param = np.hstack((pad, param))

    # If exceeds max_len keep last samples
    elif param.shape[1] > max_len:
        param = param[:, -max_len:]

    param = torch.FloatTensor(param)

    return param

In [6]:
def get_classes():
    classes = ['neg', 'pos']
    weight = None
    class_to_id = {label: i for i, label in enumerate(classes)}
    return classes, weight, class_to_id

In [7]:
class Loader(data.Dataset):
    """Data set loader::
    Args:
        root (string): Kaldi directory path.
        transform (callable, optional): A function/transform that takes in a spectrogram
            and returns a transformed version. E.g, ``transforms.RandomCrop``
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        window_size: window size for the stft, default value is .02
        window_stride: window stride for the stft, default value is .01
        window_type: typye of window to extract the stft, default value is 'hamming'
        normalize: boolean, whether or not to normalize the param to have zero mean and one std
        max_len: the maximum length of frames to use
     Attributes:
        classes (list): List of the class names.
        class_to_id (dict): Dict with items (class_name, class_index).
        wavs (list): List of (wavs path, class_index) tuples
        STFT parameters: window_size, window_stride, window_type, normalize
    """

    def __init__(self, root, transform=None, target_transform=None, window_size=.02,
                 window_stride=.01, window_type='hamming', normalize=True, max_len=1000):

        classes, weight, class_to_id = get_classes()
        self.root = root
        self.wavs = make_dataset(root, class_to_id)
        self.classes = classes
        self.weight = weight
        self.class_to_id = class_to_id
        self.transform = transform
        self.target_transform = target_transform
        self.loader = param_loader
        self.window_size = window_size
        self.window_stride = window_stride
        self.window_type = window_type
        self.normalize = normalize
        self.max_len = max_len

    def __getitem__(self, index):
        """
        Args:
            index (int): Index
        Returns:
            tuple: (key, params, target) where target is class_index of the target class.
        """
        key, path, target = self.wavs[index]
        path = '../input/covid3/wavs16k/' + path
        params = self.loader(path, self.window_size, self.window_stride, self.window_type, self.normalize, self.max_len)  # pylint: disable=line-too-long
        if self.transform is not None:
            params = self.transform(params)
        if self.target_transform is not None:
            target = self.target_transform(target)

        return key, params, target

    def __len__(self):
        return len(self.wavs)

In [8]:
class VGG(nn.Module):

    def __init__(self, vgg_name, hidden=64, dropout=0.4):
        super(VGG, self).__init__()
        self.features = make_layers(cfg[vgg_name])
        self.classifier = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(2*512, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, 1),
        )
        self._initialize_weights()

    def forward(self, x):
        x.unsqueeze_(1)
        x = self.features(x)
        x1, _ = x.max(dim=-1)
        x2 = x.mean(dim=-1)
        x = torch.cat((x1, x2), dim=-1)
        # print(x.shape)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x.squeeze(-1)

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


def make_layers(cfg, batch_norm=True):
    layers = []
    in_channels = 1
    for v in cfg:
        if v == 'M':
            layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
        else:
            conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
            if batch_norm:
                layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
            else:
                layers += [conv2d, nn.ReLU(inplace=True)]
            in_channels = v
    return nn.Sequential(*layers)


cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

In [9]:
def train(loader, model, criterion, optimizer, epoch, cuda, log_interval, weight=None, verbose=True):
    model.train()
    global_epoch_loss = 0
    samples = 0
    for batch_idx, (_, data, target) in enumerate(loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.float())
        loss.backward()
        optimizer.step()
        global_epoch_loss += loss.data.item() * len(target)
        samples += len(target)
        if verbose:
            if batch_idx % log_interval == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, samples, len(loader.dataset), 100 * samples / len(loader.dataset), global_epoch_loss / samples))
    return global_epoch_loss / samples

In [10]:
def test(loader, model, criterion, cuda, verbose=True, data_set='Test', save=None):
    model.eval()
    test_loss = 0
    tpred = []
    ttarget = []

    if save is not None:
        csv = open(save, 'wt')
        print('index,prob', file=csv)

    with torch.no_grad():
        for keys, data, target in loader:
            if cuda:
                data, target = data.cuda(), target.cuda()
            output = model(data)
            pred = output.sigmoid()
            tpred.append(pred.cpu().numpy())

            if target[0] != -1:
                loss = criterion(output, target.float()).data.item()
                test_loss += loss * len(target) # sum up batch loss 
                ttarget.append(target.cpu().numpy())

            if save is not None:
                for i, key in enumerate(keys):
                    print(f'{key},{pred[i]}', file=csv)
    
    if len(ttarget) > 0:
        test_loss /= len(loader.dataset)
        auc = roc_auc_score(np.concatenate(ttarget), np.concatenate(tpred))
        if verbose:
            print('\n{} set: Average loss: {:.4f}, AUC: ({:.1f}%)\n'.format(data_set, test_loss, 100 * auc))

        return test_loss, auc

In [11]:
args = SimpleNamespace(
    # general options
    train_path = '../input/covid3/train',        # train data folder
    valid_path = '../input/covid3/valid',        # valid data folder
    test_path = '../input/covid3/test',          # test data folder
    batch_size = 32,                             # training and valid batch size
    test_batch_size = 32,                        # batch size for testing
    arc = 'VGG13',                               # VGG11, VGG13, VGG16, VGG19
    epochs = 100,                                # maximum number of epochs to train
    lr = 0.0005,                                 # learning rate
    momentum = 0.9,                              # SGD momentum, for SGD only
    optimizer = 'adam',                          # optimization method: sgd | adam
    seed = 1234,                                 # random seed
    log_interval = 5,                            # how many batches to wait before logging training status
    patience = 5,                                # how many epochs of no loss improvement should we wait before stop training
    checkpoint = '.',                            # checkpoints directory
    train = True,                                # train before testing
    cuda = True,                                 # use gpu

    # feature extraction options
    window_size = .04,                           # window size for the stft
    window_stride = .02,                         # window stride for the stft
    window_type = 'hamming',                     # window type for the stft
    normalize = True,                            # use spect normalization
    num_workers = 2,                             # how many subprocesses to use for data loading
)

In [12]:
args.cuda = args.cuda and torch.cuda.is_available()
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)
    print('Using CUDA with {0} GPUs'.format(torch.cuda.device_count()))


# build model
model = VGG(args.arc)
if args.cuda:
    model.cuda()

# Define criterion
criterion = nn.BCEWithLogitsLoss(reduction='mean') # This loss combines a Sigmoid layer and the BCELoss in one single class.

Using CUDA with 1 GPUs


## Train model

In [13]:
# loading data
if args.train:
    train_dataset = Loader(args.train_path, window_size=args.window_size, window_stride=args.window_stride,
        window_type=args.window_type, normalize=args.normalize)
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=args.batch_size, shuffle=True,
        num_workers=args.num_workers, pin_memory=args.cuda, sampler=None)

    valid_dataset = Loader(args.valid_path, window_size=args.window_size, window_stride=args.window_stride,
        window_type=args.window_type, normalize=args.normalize)
    valid_loader = torch.utils.data.DataLoader(
        valid_dataset, batch_size=args.batch_size, shuffle=None,
        num_workers=args.num_workers, pin_memory=args.cuda, sampler=None)

    # define optimizer
    if args.optimizer.lower() == 'adam':
        optimizer = optim.Adam(model.parameters(), lr=args.lr)
    else:
        optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=args.momentum)

    best_valid_auc = 0
    iteration = 0
    epoch = 1

    # trainint with early stopping
    t0 = time.time()
    while (epoch < args.epochs + 1) and (iteration < args.patience):
        train(train_loader, model, criterion, optimizer, epoch, args.cuda, args.log_interval,
            weight=train_dataset.weight)
        valid_loss, valid_auc = test(valid_loader, model, criterion, args.cuda, data_set='Validation')
        if not os.path.isdir(args.checkpoint):
            os.mkdir(args.checkpoint)
        torch.save(model.state_dict(), './{}/model{:03d}.pt'.format(args.checkpoint, epoch))
        if valid_auc <= best_valid_auc:
            iteration += 1
            print('AUC was not improved, iteration {0}'.format(str(iteration)))
        else:
            print('Saving state')
            iteration = 0
            best_valid_auc = valid_auc
            state = {
                'valid_auc': valid_auc,
                'valid_loss': valid_loss,
                'epoch': epoch,
            }
            if not os.path.isdir(args.checkpoint):
                os.mkdir(args.checkpoint)
            torch.save(state, './{}/ckpt.pt'.format(args.checkpoint))
        epoch += 1
        print(f'Elapsed seconds: ({time.time() - t0:.0f}s)')

Train Epoch: 1 [32/2160 (1%)]	Loss: 0.693231
Train Epoch: 1 [192/2160 (9%)]	Loss: 0.693242
Train Epoch: 1 [352/2160 (16%)]	Loss: 0.691858
Train Epoch: 1 [512/2160 (24%)]	Loss: 0.693454
Train Epoch: 1 [672/2160 (31%)]	Loss: 0.693392
Train Epoch: 1 [832/2160 (39%)]	Loss: 0.692011
Train Epoch: 1 [992/2160 (46%)]	Loss: 0.691441
Train Epoch: 1 [1152/2160 (53%)]	Loss: 0.691586
Train Epoch: 1 [1312/2160 (61%)]	Loss: 0.690815
Train Epoch: 1 [1472/2160 (68%)]	Loss: 0.690531
Train Epoch: 1 [1632/2160 (76%)]	Loss: 0.689095
Train Epoch: 1 [1792/2160 (83%)]	Loss: 0.690117
Train Epoch: 1 [1952/2160 (90%)]	Loss: 0.689751
Train Epoch: 1 [2112/2160 (98%)]	Loss: 0.689800

Validation set: Average loss: 0.6840, AUC: (61.6%)

Saving state
Elapsed seconds: (80s)
Train Epoch: 2 [32/2160 (1%)]	Loss: 0.679039
Train Epoch: 2 [192/2160 (9%)]	Loss: 0.697365
Train Epoch: 2 [352/2160 (16%)]	Loss: 0.692832
Train Epoch: 2 [512/2160 (24%)]	Loss: 0.692115
Train Epoch: 2 [672/2160 (31%)]	Loss: 0.691173
Train Epoch: 2 [8

## Test Model

In [14]:
test_dataset = Loader(args.test_path, window_size=args.window_size, window_stride=args.window_stride,
    window_type=args.window_type, normalize=args.normalize)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=args.test_batch_size, shuffle=None,
    num_workers=args.num_workers, pin_memory=args.cuda, sampler=None)

# get best epoch
state = torch.load('./{}/ckpt.pt'.format(args.checkpoint))
epoch = state['epoch']
print("Testing model (epoch {})".format(epoch))
model.load_state_dict(torch.load('./{}/model{:03d}.pt'.format(args.checkpoint, epoch)))
if args.cuda:
    model.cuda()

results = 'submission.csv'
print("Saving results in {}".format(results))
test(test_loader, model, criterion, args.cuda, save=results)

Testing model (epoch 13)
Saving results in submission.csv
